In [ ]:
import pandas as pd
import numpy as np
from garmin_fit_sdk import Decoder, Stream
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import glob
from datetime import datetime
from scipy import signal
# from util import lowpass_filter
FIT_EPOCH_S = 631065600

In [5]:
%pwd

'/home/d0ugins/cmu/ec/srs/backend/src/notebooks'

In [11]:
file_list = sorted(glob.glob("../../../data/virbs/all/*.fit"))
file_list

['../../../data/virbs/all/2025-09-14-08-31-56.fit',
 '../../../data/virbs/all/2025-09-14-08-32-01.fit',
 '../../../data/virbs/all/2025-09-20-07-27-45.fit',
 '../../../data/virbs/all/2025-09-20-07-30-37.fit',
 '../../../data/virbs/all/2025-09-20-07-49-28.fit',
 '../../../data/virbs/all/2025-09-20-07-50-11.fit',
 '../../../data/virbs/all/2025-09-20-08-18-07.fit',
 '../../../data/virbs/all/2025-09-20-08-18-20.fit',
 '../../../data/virbs/all/2025-09-20-08-19-55.fit',
 '../../../data/virbs/all/2025-09-20-08-33-59.fit',
 '../../../data/virbs/all/2025-09-20-08-34-14.fit',
 '../../../data/virbs/all/2025-09-20-08-56-21.fit',
 '../../../data/virbs/all/2025-09-21-07-10-41.fit',
 '../../../data/virbs/all/2025-09-21-07-27-14.fit',
 '../../../data/virbs/all/2025-09-21-07-28-54.fit',
 '../../../data/virbs/all/2025-09-21-07-43-36.fit',
 '../../../data/virbs/all/2025-09-21-07-56-07.fit',
 '../../../data/virbs/all/2025-09-21-07-56-40.fit',
 '../../../data/virbs/all/2025-09-21-08-12-29.fit',
 '../../../d

In [18]:
# file_name = "2025-09-14-07-13-58"
# file_name = file_list[15]
file_name="../../../data/virbs/all/2025-11-02-07-11-57.fit"
stream = Stream.from_file(file_name)
decoder = Decoder(stream)
messages, errors = decoder.read(convert_datetimes_to_dates=False)
print(file_name)
errors

../../../data/virbs/all/2025-11-02-07-11-57.fit


[]

In [13]:
starts = [m for m in messages['camera_event_mesgs'] if m['camera_event_type'] == 'video_start']
if len(starts) != 1:
    raise ValueError(f"Found {len(starts)} video start events")
start = starts[0]
start_time = start['timestamp'] * 1000 + start['timestamp_ms']
start_time

3272

In [14]:
gps_mesgs = messages['gps_metadata_mesgs']
gps_data = pd.DataFrame.from_records(gps_mesgs)
gps_data['utc_timestamp'] = pd.to_datetime((gps_data.utc_timestamp + FIT_EPOCH_S) * 1e9)
gps_data.index = (gps_data.timestamp * 1000 + gps_data.timestamp_ms) - start_time
gps_data.head()

,timestamp,position_lat,position_long,enhanced_altitude,enhanced_speed,utc_timestamp,timestamp_ms,heading,velocity,8,9,10,11,12
24894,28,482489466,-953741519,327.2,0.545,2025-11-02 12:29:09,166,124.98,"[0.44, -0.31, -0.04]",229,90,246,95,106
24994,28,482489289,-953741669,329.0,0.632,2025-11-02 12:29:09,266,127.38,"[0.5, -0.38, -0.04]",226,88,243,95,106
25094,28,482489177,-953741730,329.8,0.689,2025-11-02 12:29:09,366,130.11,"[0.52, -0.44, -0.04]",224,85,240,95,106
25194,28,482489097,-953741761,330.2,0.744,2025-11-02 12:29:09,466,131.60,"[0.55, -0.49, -0.04]",222,83,237,95,106
25294,28,482489029,-953741787,330.4,0.791,2025-11-02 12:29:09,566,133.69,"[0.57, -0.54, -0.04]",220,82,235,95,106


In [9]:
calibration_mesgs = messages['three_d_sensor_calibration_mesgs']
calibration_data = { m['sensor_type']: m for m in calibration_mesgs }
accel_cal = calibration_data['accelerometer']
accel_cal

{'timestamp': 2,
 'calibration_factor': 1,
 'calibration_divisor': 2048,
 'level_shift': 32768,
 'offset_cal': [34, -9, -103],
 'orientation_matrix': [0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0],
 'sensor_type': 'accelerometer',
 'accel_cal_factor': 1}

In [10]:
accel_raw_list = []
for group in messages['accelerometer_data_mesgs']:
    base_timestamp = group['timestamp'] * 1000 + group['timestamp_ms']
    for i, offset in enumerate(group['sample_time_offset']):
        entry = {
            'timestamp': base_timestamp + offset,
            'x': group['accel_x'][i],
            'y': group['accel_y'][i],
            'z': group['accel_z'][i],
        }
        accel_raw_list.append(entry)
accel_raw = pd.DataFrame.from_records(accel_raw_list)
accel_raw = accel_raw.set_index('timestamp').sort_index()
accel_raw.index = accel_raw.index - start_time

In [11]:
accel_data = np.array(accel_cal['orientation_matrix']).reshape(3, 3) @ ((accel_raw.to_numpy() \
- accel_cal['level_shift'] - accel_cal['offset_cal']) * \
(accel_cal['calibration_factor'] / accel_cal['calibration_divisor'])).T
accel_data = accel_data.T
accel_data = pd.DataFrame(accel_data, columns=['x', 'y', 'z'], index=accel_raw.index)
fs = 1000 / np.mean(np.diff(accel_data.index))
accel_data['magnitude'] = np.sqrt(accel_data.x**2 + accel_data.y**2)


accel_filtered = pd.DataFrame(index=accel_data.index, data={
    'x': lowpass_filter(accel_data.x.to_numpy(), 5, fs),
    'y': lowpass_filter(accel_data.y.to_numpy(), 5, fs),
    'z': lowpass_filter(accel_data.z.to_numpy(), 5, fs),
})
accel_filtered['magnitude'] = np.sqrt(accel_filtered.x**2 + accel_filtered.y**2)

In [12]:
# 4.986, 14.411
px.line(gps_data.enhanced_speed)

In [43]:
px.line(accel_filtered.magnitude)

In [40]:


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=gps_data.index, y=gps_data.enhanced_speed, name="v"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=accel_filtered.index, y=accel_filtered.magnitude, name="a"),
    secondary_y=True,
)
fig.show()


In [62]:
f, Pxx = signal.welch(accel_filtered.magnitude.to_numpy(), fs, nperseg=1024)
px.line(pd.DataFrame(index=f, data=dict(PSD=Pxx)))

In [19]:
import json
with open(f'../../../tmp/{file_name.split('/')[-1].replace(".fit", ".json")}', 'w') as f:
    json.dump(messages, f, indent=2, default=str)